## Here we generate training, validation and test dataset. with window_size = 8192

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pickle 
import random

import sys
sys.path.append('../../src')
import data_utils_synthetic as data_utils

In [2]:
#load the data
data_path_train = '../../data/synthetic2/data_8192_1000_3_4_4_.03_.03__.00-.05.mat'
data_path_test = '../../data/synthetic2/data_8192_1000_3_4_4_.03_.03__.10-.15.mat'

data_raw_train = data_utils.load_mat(data_path_train, 'data')
data_raw_test = data_utils.load_mat(data_path_test, 'data')

In [3]:
data_raw_train.shape, data_raw_test.shape

((1000, 3, 4, 8192), (1000, 3, 4, 8192))

In [4]:
#reshape data to transform 4 variables of 3 modalities of a time series to 12 variables.
data_reshaped_train = np.reshape(data_raw_train, (data_raw_train.shape[0], -1, data_raw_train.shape[-1]))
data_reshaped_test = np.reshape(data_raw_test, (data_raw_test.shape[0], -1, data_raw_test.shape[-1]))
#reshape to have (batch_size, signal_len, num_vari)
data_transposed_train = np.transpose(data_reshaped_train, (0, 2, 1))
data_transposed_test = np.transpose(data_reshaped_test, (0, 2, 1))

#parameters
window_size = 8192
stride = 8192

train_idx = np.arange(0,900)
validation_idx = np.arange(900,1000)


data_raw_train = data_transposed_train[train_idx]
data_raw_valdation = data_transposed_train[validation_idx]
data_raw_test = data_transposed_test

_, data_stacked_train = data_utils.slide_window(data_raw_train,
                                                    window_size,
                                                    stride,
                                                    num_dim_expand=0)

_, data_stacked_validation = data_utils.slide_window(data_raw_valdation,
                                                    window_size,
                                                    stride,
                                                    num_dim_expand=0)

_, data_stacked_test = data_utils.slide_window(data_raw_test,
                                               window_size,
                                               stride,
                                               num_dim_expand=0)

data_stacked_train.shape, data_stacked_validation.shape, data_stacked_test.shape

((900, 8192, 12), (100, 8192, 12), (1000, 8192, 12))

In [5]:
data_utils.load_mat(data_path_train, 'gtruth')[train_idx].shape

(900, 8192)

In [6]:
gt_raw_train = data_utils.load_mat(data_path_train, 'gtruth')[train_idx]
gt_raw_validation = data_utils.load_mat(data_path_train, 'gtruth')[validation_idx]
gt_raw_test = data_utils.load_mat(data_path_test, 'gtruth')

_, gt_stacked_train = data_utils.slide_window(gt_raw_train,
                                              window_size,
                                              stride,
                                              num_dim_expand=0)

_, gt_stacked_validation = data_utils.slide_window(gt_raw_validation,
                                              window_size,
                                              stride,
                                              num_dim_expand=0)

_, gt_stacked_test = data_utils.slide_window(gt_raw_test,
                                             window_size,
                                             stride,
                                             num_dim_expand=0)

gt_stacked_train.shape, gt_stacked_validation.shape, gt_stacked_test.shape

((900, 8192), (100, 8192), (1000, 8192))

In [7]:
#index of changes
gt_mean_test = data_utils.load_mat(data_path_test, 'gt_mean')
gt_mean_test.shape

(1000, 4)

In [8]:
data = {}
data['train_data'] = data_stacked_train
data['train_gt'] = gt_stacked_train

data['validation_data'] = data_stacked_validation
data['validation_gt'] = gt_stacked_validation

data['test_data'] = data_stacked_test
data['test_gt'] = gt_stacked_test

data['index_of_changes'] = gt_mean_test

In [9]:
path_to_save = '../../data/synthetic2/'
with open(path_to_save+'processed2.p', 'w') as fout:
    pickle.dump(data, fout)